# Generating and managing inputs

:::{admonition} Learning Objectives
:class: learning-objectives

For larger and more complex projects it is beneficial to automate the generation of input nodes, rather than manually creating them.
We would also like to be able to manage the inputs in a structured way, so that we can easily find them and change them.

In this section we will look at methods for generating input nodes in a more automated fashion, and how to manage them using groups and extras.

:::

In the previous section, we generated the inputs for a calculation manually,
by reading from a previously written input file.

For more complex calculations though, it is likely that we would like to automate the input generation process.

In this section, we will see the some methods for generating the inputs for a `PwCalculation` automatically.

In [ ]:
from local_module import load_temp_profile

data = load_temp_profile(name="input-gen", add_sssp=True, add_structure_si=True)
data

## Importing structures from external databases

AiiDA ships with a number of plugins that allow you to import structures from external databases.
These plugins are called `importers` and are available in the `aiida.tools.dbimporters` module.

As an example, let's import a structure from the [Crystallography Open Database (COD)](http://www.crystallography.net/cod/).

In [ ]:
from aiida.tools.dbimporters.plugins.cod import CodEntry

entry = CodEntry("http://www.crystallography.net/cod/1526655.cif")
structure = entry.get_aiida_structure()
structure.get_pymatgen()

AiiDA also works closely with the [Open Databases Integration for Materials Design (OPTIMADE) consortium](https://www.optimade.org/),
which provides a common REST API for querying a number of databases for structures.

![optimade](_static/aiida/optimade.png){align=center width=350px}

In [ ]:
from local_module.optimade import count_structures, get_providers, yield_structures

get_providers()[["name", "description"]]

We can also use the OPTIMADE plugin to query the COD database for a structure.

In [ ]:
count_structures("cod", 'chemical_formula_hill = "Si" AND nelements = 1')

In [ ]:
# for structure in yield_structures("cod", 'chemical_formula_hill = "Si"', max_results=2):
#     print(structure)

## Importing Pseudopotential families

The [aiida-pseudo](https://github.com/aiidateam/aiida-pseudo) package provides a number of pseudopotential families that are available for download.
These install a full set of pseudopotentials for all elements into the AiiDA profile as a `Group`.

```console
$ aiida-pseudo install sssp -x PBE -p efficiency -v 1.1
```

They can later be loaded:

In [ ]:
from aiida import orm

family = orm.load_group(label="SSSP/1.1/PBE/efficiency")
family

## Using Groups

We can view all groups in our profile using the `verdi group list` command:

In [ ]:
%verdi group list -a --all

Groups are a powerful feature of AiiDA, allowing you to organize your data in a flexible way.
They can be used to organize nodes into sets, or to assign nodes as members of subgroups.
You can add any nodes to a group, and any node can be in multiple groups.

Lets create a new group



In [ ]:
%verdi group create my_group

Now we can add a set of structures to the group:

In [ ]:
group = orm.load_group("my_group")

other_structure = data.structure.clone().store()

for structure in [data.structure, other_structure]:
    structure.store()
    group.add_nodes(structure)

In [ ]:
%verdi group show my_group

## Adding extras to nodes and querying for them

We can add extra information to nodes using the `extras` dictionary.
This is a flexible way to add additional information to nodes that is not part of the node's provenance model,
i.e. these can be modified after the node is stored (as opposed to other data).

In [ ]:
group = orm.load_group("my_group")
node: orm.Node
for node in group.nodes:
    node.base.extras.set('structure', 'silicon')

AiiDA provides a powerful query system that allows you to search for nodes based on their properties,
and location in the provenance graph.

We can gather all the structures we tagged as `silicon` using the `QueryBuilder`:


In [ ]:
query = orm.QueryBuilder()
query.append(orm.Group, filters={"label": "my_group"}, tag="group")
query.append(
    orm.StructureData,
    with_group="group",
    tag="structure",
    filters={"extras.structure": "silicon"},
    project="*",
)
query.all(flat=True)

:::{seealso}

The [Query Builder documentation](https://aiida.readthedocs.io/projects/aiida-core/en/latest/howto/query.html) provides more information on how to use the `QueryBuilder`,
and the features available.

:::